<a href="https://colab.research.google.com/github/BarelHeby/Deep-Learning---Chest-X-Ray/blob/main/Deep_Learning_Chest_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---
**FIRST SETUP**

In [ ]:
!pip install -q kaggle
from google.colab import files

# Create a Kaggle API token and save it to a file.
!echo '{"username":"barelheby","key":"978bdd6cd1cc991c69bbe920fe75a9cc"}' > kaggle.json

# Copy the Kaggle API token file to the correct location.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json

# Change the permissions of the Kaggle API token file.
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset from Kaggle.
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

# Unzip the downloaded dataset.
!unzip chest-xray-pneumonia.zip

# Print the contents of the current working directory.
!ls


---
**IMPORTS**

In [ ]:
import pandas as pd
from os.path import join
import tensorflow as tf



---
 **Configuration**


In [ ]:
TF_BATCH_SIZE = 32
TF_SEED = 123
IMG_HEIGHT = 256
IMG_WIDTH = 256
DIR_DATASET = "/content/chest_xray"

---
**Normalize Image Function**



---

**Training Set Preprocess**

In [ ]:
train_directory = join(DIR_DATASET,"train")
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    validation_split=0.2,
    subset = "training",
    seed = TF_SEED,
    image_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = TF_BATCH_SIZE,
)

Found 5216 files belonging to 2 classes.
Using 4173 files for training.


---

**Validation Set Prep**

In [ ]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    validation_split=0.2,
    subset = "validation",
    seed = TF_SEED,
    image_size = (IMG_HEIGHT,IMG_WIDTH),
    batch_size = TF_BATCH_SIZE
)

Found 5216 files belonging to 2 classes.
Using 1043 files for validation.


In [ ]:
train_dataset.class_names

['NORMAL', 'PNEUMONIA']

In [ ]:
train_encoded_class_names = [i for (i,label) in enumerate(train_dataset.class_names)]
validation_encoded_class_names = [i for (i,label) in enumerate(validation_dataset.class_names)]


In [ ]:
train_dataset = train_dataset.map(lambda x,y:(x/255.0,y))
validation_dataset = validation_dataset.map(lambda x,y:(x/255.0,y))

---
**Part 1 Model**

In [ ]:
from keras import Sequential
from tensorflow.keras.applications import VGG16
from keras.layers import BatchNormalization,Dropout,Dense,Flatten
from tensorflow.keras.optimizers import Adamax,Adam


In [ ]:
base_network = VGG16(weights="imagenet",include_top=False,input_shape = (IMG_HEIGHT,IMG_WIDTH,3),pooling="avg")
base_network.trainable = False

In [ ]:
model  = Sequential(name="Assigment_1")
model.add(base_network)
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.50))
# model.add(Dense(220,activation="relu"))
# model.add(Dropout(0.25))
# model.add(Dense(60,activation="relu"))
model.add(Dense(1,activation="softmax"))
model.compile(optimizer=Adam(),loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "Assigment_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_10 (Flatten)        (None, 512)               0         
                                                                 
 dense_43 (Dense)            (None, 512)               262656    
                                                                 
 dropout_25 (Dropout)        (None, 512)               0         
                                                                 
 dense_44 (Dense)            (None, 1)                 513       
                                                                 
Total params: 14977857 (57.14 MB)
Trainable params: 263169 (1.00 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_dataset,
                        epochs= 10,
                        validation_data = validation_dataset,
                        callbacks = early_stopping)

Epoch 1/10
131/131 [==============================] - 40s 282ms/step - loss: 0.3447 - accuracy: 0.7345 - val_loss: 0.2067 - val_accuracy: 0.7766
Epoch 2/10
131/131 [==============================] - 38s 281ms/step - loss: 0.2018 - accuracy: 0.7345 - val_loss: 0.1306 - val_accuracy: 0.7766
Epoch 3/10
131/131 [==============================] - 42s 309ms/step - loss: 0.1574 - accuracy: 0.7345 - val_loss: 0.1098 - val_accuracy: 0.7766
Epoch 4/10
131/131 [==============================] - 40s 296ms/step - loss: 0.1374 - accuracy: 0.7345 - val_loss: 0.1003 - val_accuracy: 0.7766
Epoch 5/10
 26/131 [====>.........................] - ETA: 23s - loss: 0.1218 - accuracy: 0.7560

KeyboardInterrupt: 

In [ ]:
device_name = tf.test.gpu_device_name()
device_name

'/device:GPU:0'

In [ ]:
validation_loss, validation_accuracy = model.evaluate(validation_dataset)

33/33 [==============================] - 8s 182ms/step - loss: 0.0647 - accuracy: 0.7766


In [ ]:
print("Validation Loss:", validation_loss)
print("Validation Accuracy:", validation_accuracy)

Validation Loss: 0.0647210106253624
Validation Accuracy: 0.7766059637069702
